In [ ]:
from pinecone import Pinecone, ServerlessSpec

from transformers import CLIPProcessor, CLIPModel

# Load the model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

API_KEY = "pcsk_V1N5k_2MEH2y1B7ZwGjYKbHXw5twJCGpiyPZ4eEg6GmESWrGVmApeBp6q4gxgNNLYHd9P"

client = Pinecone(
    api_key=API_KEY,
)


In [46]:
import json
with open("products.json" , "r") as file:
    data = json.load(file)
    

data = data[:40]

In [ ]:
client.delete_index("products")
client.create_index("products" , dimension=512 , metric = "cosine" , spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) )

index = client.Index("products")

In [47]:
for i in index.list():
    print(i)

In [73]:
import requests
from io import BytesIO
from PIL import Image
import torch
vectors = []
id = 1
i = 1
for d in data:
    images = d["images"]
    for image in images:
        res = requests.get(image)
        image_ = Image.open(BytesIO(res.content))

        inputs = processor(images=image_, return_tensors="pt")
        with torch.no_grad():
            image_embeddings = list(model.get_image_features(**inputs))
        
        category_name = d.get("category_name" , "")
        category_id = str(d.get("category_id" , ""))
        shop_id = str(d.get("shop_id" , ""))
        shop_name = d.get("shop_name" , "")
        link = d.get("link" , "")
        status = d.get("status" , "")
        region = d.get("region" , "")
        images = d.get("images" , "")
        
        vectors.append({
                "values" : image_embeddings[0],
                "id" : str(id),
                "metadata" : {
                    "name" : d["name"],
                    "pid" : str(d["id"]),
                    "category_name" :category_name or "" ,
                    "category_id" : category_id or "",
                    "shop_id":  shop_id or "",
                    "shop_name":  shop_name or "",
                    "link":  link or "",
                    "status":  status or "",
                    "region" : region or "",
                    "images" : images or []
                }
            })
        id +=1
        break
    print(i)
    i+=1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


In [76]:
index.upsert(vectors)

{'upserted_count': 40}

In [79]:
def fetch_relevant_vector(text_description, top_k=10, namespace=None , filters = {}):
      # Process the text input to get embeddings
      inputs = processor(text=[text_description], return_tensors="pt", padding=True)
      with torch.no_grad():
           text_embedding = model.get_text_features(**inputs)
      text_embedding = text_embedding.squeeze().tolist() 
      
      response = index.query(
           namespace=namespace,
           vector=text_embedding,
           top_k=top_k,
           include_values=True,
           include_metadata=True,
           filter={"status" : "IN_STOCK"}
           )
 
      return response

In [ ]:
result = fetch_relevant_vector("zebra")

matches = result["matches"]
# print(ma)
aaa = []
for m in matches:
    aaa.append(m["metadata"]["pid"])


0.271046281
0.252119809
0.243664354
0.235029012
0.216721714
0.216632858
0.2144133
0.212330014
0.211035833
0.210574493
{'2058607', '2089195', '2089165', '2089194', '2089192', '2089198', '2089197', '2089171', '2089199', '2089193'}
